# Serve Llama2-70B on SageMaker using DJL container

In this notebook, we deploy the Llama-2-70B model across GPUs on a ml.p4d.24xlarge instance.

## Import the relevant libraries and configure several global variables using boto3

In [ ]:
!pip install sagemaker boto3 --upgrade

In [1]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "hf-large-model-djl/code_llama2"  # folder within bucket where code artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Create SageMaker compatible Model artifact, upload model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties`.

The tarball is in the following format:

```
code
├──── 
│   └── serving.properties

```

- `serving.properties` is the configuration file that can be used to configure the model server.

In [3]:
!mkdir -p code_llama2

#### Create serving.properties 
This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -
- `engine`: The engine for DJL to use. In this case, we intend to use Accelerate and hence set it to **Python**. 
- `option.model_id`: The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co.
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which Accelerate needs to partition the model. This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.

In [4]:
# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/llama2/"
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

Pretrained model will be downloaded from ---- > s3://sagemaker-example-files-prod-us-west-2/models/llama2/


In the below cell, we leverage [Jinja](https://pypi.org/project/Jinja2/) to create a template for serving.properties. Specifically, we parameterize `option.s3url` so that it can be changed based on the pretrained model location.

In [68]:
%%writefile ./code_llama2/serving.properties
engine=MPI
#option.model_id=TheBloke/Llama-2-70B-fp16
option.s3url=s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_llama2_model/
option.tensor_parallel_degree=8
option.dtype=fp16
option.paged_attention=true
option.rolling_batch=auto
option.max_rolling_batch_size=8
option.max_rolling_batch_prefill_tokens=1000
option.model_loading_timeout=3600

Overwriting ./code_llama2/serving.properties


In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("code_llama2/serving.properties").open().read())
Path("code_llama2/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize code_llama2/serving.properties | cat -n

**Image URI for the DJL container is being used here**

In [70]:
# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


**Create the Tarball and then upload to S3 location**

In [71]:
!rm -f model.tar.gz
!tar czvf model.tar.gz -C code_llama2 .

./
./serving.properties


In [72]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/hf-large-model-djl/code_llama2/model.tar.gz


### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.p4d.24xlarge

    b) ContainerStartupHealthCheckTimeoutInSeconds is 3600 to ensure health check starts after the model is ready    


3. Create the end point using the endpoint config created

#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the container because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.

In [73]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"llama2-djl")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri,
                      "ModelDataUrl": s3_code_artifact,
                      'Environment': {'MODEL_LOADING_TIMEOUT': '3600'}
                     }
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

llama2-djl-2023-09-11-14-32-41-661
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/llama2-djl-2023-09-11-14-32-41-661


In [74]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.48xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/llama2-djl-2023-09-11-14-32-41-661-config',
 'ResponseMetadata': {'RequestId': '11e1154b-caae-4cab-8943-f04e5fbec222',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11e1154b-caae-4cab-8943-f04e5fbec222',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '122',
   'date': 'Mon, 11 Sep 2023 14:32:43 GMT'},
  'RetryAttempts': 0}}

In [75]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/llama2-djl-2023-09-11-14-32-41-661-endpoint


### This step can take ~ 20 min or longer so please be patient

In [76]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/llama2-djl-2023-09-11-14-32-41-661-endpoint
Status: InService


#### Leverage Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results.

You can pass a prompt as input to the model. This done by setting `inputs` to a prompt. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters. These `parameters` need to be passed to the endpoint as a dictionary of `kwargs`. Refer this documentation - https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig for more details.

The below code sample illustrates the invocation of the endpoint using a text prompt and also sets some parameters.

In [99]:
%%time
prompt1 = """根据以下反引号内的商品详细描述，为电商直播主持人创作一段引人注目的商品推介话术
‘’‘
iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。
后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素
此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间
’‘’
话术中应包括商品的主要特点、优势及互动环节,使用中文撰写，并保持话术简洁、有趣且具吸引力,并确保包含上述要求的所有元素"""

prompt2="""根据以下反引号内的关键词，为电商直播主持人创作一段通用的开场、互动或欢迎话术。请确保话术融入这些关键词，使用中文撰写，内容要简洁、有趣且具吸引力，同时适应广泛的商品和场景。
‘’‘
精选
性价比
品质
日常家居
穿搭
限时折扣
免费赠品
抽奖活动
大品牌合作
独家优惠
’‘’
请使用上述关键词，编写一段具有普遍适用性，适于电商直播开头或互动环节的话术"""


prompt3="""
请根据以下反引号内的商品描述、意图、问题模板和回答模板，为电商直播商品生成一个问答库。要求生成的回答应当有至少一组，最多五组。请确保答案基于商品描述和回答模板生成。如果无法生成回答，表示为“根据已知信息无法生成回答”。格式应如下：{{"Q":"问题","A":['答案1-1','答案1-2'...]}}
‘’‘
商品描述：iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素。此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间。}
意图：性能
问题模板：手机的性能如何？
回答模板：[商品名称]采用了最新的[芯片名称]，搭载了[核心数量]核CPU和[GPU核心数量]核GPU，为用户提供强大的性能。
谈到[商品名称]的性能，不得不提及它的[芯片名称]，配备[核心数量]核CPU和[GPU核心数量]核GPU，应对各种任务都游刃有余。
[商品名称]在性能上表现卓越，得益于其[芯片名称]和[核心数量]核处理器，加上[GPU核心数量]核GPU，让每次使用都顺畅无比。}
’‘’
问答生成：请基于上述商品描述、意图、问题模板和回答模板，为电商直播商品提供符合上述格式的问答库。
"""

prompt4="""
请以电商直播主持人的第一人称角度回答观众的商品相关问题。确保只回答与商品相关的问题，并只使用以下反引号内知识库的信息来回答。回答中请勿随意编造内容。格式应如下:[{{"intention": "意图1", "answer": "回答1"}},{{"intention": "意图2", "answer": "回答2"}}]
‘’‘
[问题：iPhone 14有哪些可选的颜色？][回答：iPhone 14提供了六种时尚的颜色选择，包括蓝色、紫色、午夜色、星光色、红色和黄色。][意图：颜色]
[问题：关于摄像头，iPhone 14的前置和后置摄像头分辨率是多少？][回答：iPhone 14的前置和后置摄像头分辨率都是1200万像素。][意图：分辨率]
[问题：我经常用手机办公和玩游戏，iPhone 14的性能如何？][回答：iPhone 14搭载了强大的苹果A15六核中央处理器，无论是玩游戏、看视频，还是办公，它都可以轻松应对。][意图：性能]}
’‘’
观众问题：主播小姐姐好漂亮
使用第一人称直接回答观众关于商品的提问。检查知识库中是否有与观众提问相匹配的回答。对于在知识库中找到的每个匹配意图，请依次提供对应的回答，并确保从知识库中的意图中提取相应的意图标签。如果所有的意图都在知识库中找不到答案，回答“根据已知信息无法回答问题”。确保不使用emoji。
"""

other="""我很在意手机的颜色和摄像头功能，能给我介绍一下iPhone 14在这两方面的特点吗？
便宜点就好了"""
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": prompt3,
            "parameters": {
                "max_new_tokens": 500,
                "do_sample": False,
            },
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("unicode_escape")

CPU times: user 2.87 ms, sys: 1.19 ms, total: 4.06 ms
Wall time: 12.1 s


'{"generated_text": "onyoung瓷砖视频支付攻击者远程控制vps-links-podeba-and-cve-2021-44228\n去年文革之前的中国，stanford computer science分课答案\nThe Communications Engineering Professionals\nArchives Select Month December 2022 November 2022 October 2022 September 2022 August 2022 July 2022 June 2022 May 2022 April 2022 March 2022 February 2022 January 2022 December 2021 November 2021 October 2021 September 2021 August 2021 July 2021 June 2021 May 2021 April 2021 March 2021 February 2021 January 2021 December 2020 November 2020 October 2020 September 2020 August 2020 July 2020 June 2020 May 2020 April 2020 March 2020 February 2020 January 2020 December 2019 November 2019 October 2019 September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 December 2017 November 2017 October 2017 September 2017 August 2017 July 2017 June 

In [98]:
%%time
from joblib import Parallel, delayed
prompts = [prompt1,prompt2,prompt3,prompt4]

def call_endpoint(prompt):
    input = {"inputs": prompt, "parameters": parameters}
    input = json.dumps(input)

    response = smr_client.invoke_endpoint(EndpointName=endpoint_name,
                                       ContentType='application/json',
                                       Accept='application/json',
                                       Body=input)
    results = response['Body'].read().decode("unicode_escape")
    print(results)
    print("output lenth=="+str(len(results)))


results = Parallel(n_jobs=6, prefer='threads', verbose=1)(
    delayed(call_endpoint)(prompt)
    for prompt in prompts
)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


{"generated_text": "，以便被用户评估。
(Including the keywords and description requirements)
<<精选产品,得到高价再底价,品质最高可以实现自己追星追光的理想!适合男女老少,现在市面雅言百变,你是存在中生迫求的人民是“龙骑士吗?，消费者就是龙骑士了,以个人标准,为公众服务,从“龙骑士”摸底起!走进职业?底列旗舰料理厨艺,作为爱吃饭和料理的大爷雷,帮个好师兄,曾经是武道家缘恳兼,一句话是“削削梨梨”和奶对,每个料秘艺体会在自动,日色日光日日日自由,配头"}
output lenth==266
{"generated_text": "注意: 作为反引号内部的回答信息已按原句结构移植，故不使用文末、补全问号等punctuation，且均为一个独立的语句。
# 颜色
[问题: iPad Pro有哪些颜色？]
[答案: iPad Pro提供了板面金属和基本黑的两种颜色。]
# 性能
[问题: QuickPath 键盘是通过什么理念而设计的？能够支持对屏幕占满大小的键盘？]
[答案: Apple设计了一个源源不断更新的板面型键盘，它完全集成在硬件层级，并将开关时间缩短为30ms，
提供了首个同步时间接触键盘的用户体"}
output lenth==263
{"generated_text": "Current Best Best Solo Fighter Sword Art Online: Hollow Realization Deals 2022
Cheapest Best Solo Fighter Sword Art Online: Hollow Realization Deals 2022
BESTSELLER NO. 2 The Six Shooter
"For six shooters on the open trail, here's a tale to be recalled, with one brain-boiling yarn from the mighty pen that you can't compare with or recall!" ...
"Even upon the driest eye, tears come easy when you turn me on. So 

[Parallel(n_jobs=6)]: Done   4 out of   4 | elapsed:    9.6s finished


## Conclusion
In this post, we demonstrated how to use SageMaker large model inference containers to host Llama2-70B. For more details about Amazon SageMaker and its large model inference capabilities, refer to the following:

* Model parallelism and large model inference on Sagemaker (https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html)

## Clean Up

In [ ]:
# Delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# In case the endpoint failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)